## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-21-20-49-06 +0000,bbc,Gatwick second runway plan approved by transpo...,https://www.bbc.com/news/articles/c9v7rz24z23o...
1,2025-09-21-20-48-52 +0000,nyt,"Britain, Australia, Canada and Portugal Recogn...",https://www.nytimes.com/2025/09/21/world/europ...
2,2025-09-21-20-48-10 +0000,wapo,Live updates: Charlie Kirk memorial draws thou...,https://www.washingtonpost.com/politics/2025/0...
3,2025-09-21-20-48-10 +0000,wapo,Stephen Miller warns of ‘army that they have a...,https://www.washingtonpost.com
4,2025-09-21-20-43-00 +0000,wsj,Top Conservatives Remember Charlie Kirk as Chr...,https://www.wsj.com/us-news/charlie-kirk-funer...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,33
15,kirk,13
14,charlie,12
11,palestinian,11
97,he,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
2,2025-09-21-20-48-10 +0000,wapo,Live updates: Charlie Kirk memorial draws thou...,https://www.washingtonpost.com/politics/2025/0...,78
42,2025-09-21-17-51-06 +0000,nypost,Charlie Kirk funeral hits capacity as 200K peo...,https://nypost.com/2025/09/21/us-news/charlie-...,76
21,2025-09-21-19-49-06 +0000,nypost,Trump gives shock compliment to AOC as he reve...,https://nypost.com/2025/09/21/us-news/trump-gi...,64
17,2025-09-21-20-00-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...,64
148,2025-09-21-01-22-42 +0000,nypost,Elon Musk resurfaces Harris tweet calling for ...,https://nypost.com/2025/09/20/us-news/elon-mus...,61


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
2,78,2025-09-21-20-48-10 +0000,wapo,Live updates: Charlie Kirk memorial draws thou...,https://www.washingtonpost.com/politics/2025/0...
77,51,2025-09-21-13-36-12 +0000,nypost,"Canada, UK and Australia all recognize Palesti...",https://nypost.com/2025/09/21/world-news/canad...
103,40,2025-09-21-10-13-45 +0000,nypost,Crazed gunman shouted ‘Free Palestine’ before ...,https://nypost.com/2025/09/21/us-news/alleged-...
46,31,2025-09-21-17-12-42 +0000,nypost,Man charged with shooting up Sacramento ABC st...,https://nypost.com/2025/09/21/us-news/fbi-arre...
21,31,2025-09-21-19-49-06 +0000,nypost,Trump gives shock compliment to AOC as he reve...,https://nypost.com/2025/09/21/us-news/trump-gi...
17,31,2025-09-21-20-00-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...
29,25,2025-09-21-18-50-48 +0000,nypost,Trump warns US will defend Poland and Baltics ...,https://nypost.com/2025/09/21/us-news/trump-wa...
161,22,2025-09-20-23-11-48 +0000,cbc,White House scrambles to clarify Trump plan to...,https://www.cbc.ca/news/world/trump-travel-ban...
76,22,2025-09-21-13-38-54 +0000,nypost,Trump presses Bondi to prosecute his enemies i...,https://nypost.com/2025/09/21/us-news/trump-pr...
80,21,2025-09-21-13-03-34 +0000,bbc,Lib Dems call for bank windfall tax to fund gr...,https://www.bbc.com/news/articles/cvg00k8nwgwo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
